# Détection de Somnolence du Conducteur - Exploration et Préparation

## Vision par Ordinateur et Deep Learning

Ce notebook couvre:
1. **Exploration des données** - Visualisation des datasets
2. **Prétraitement** - Normalisation, augmentation
3. **Extraction de features** - EAR, MAR, Landmarks
4. **Préparation des données** - Création des jeux train/validation/test

## 1. Configuration et Imports

In [ ]:
# Imports standards
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from tqdm import tqdm
import yaml

# Configuration des visualisations
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
%matplotlib inline

# Ajout du path source
sys.path.append('../src')

# Imports du projet
from utils.preprocessing import ImagePreprocessor
from utils.metrics import calculate_ear, calculate_mar, FatigueMetrics
from detection.face_detector import FaceDetector
from detection.landmark_extractor import LandmarkExtractor

print("✓ Imports réussis")
print(f"OpenCV version: {cv2.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## 2. Chargement de la Configuration

In [ ]:
# Chargement de la configuration
with open('../config.yaml', 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

print("Configuration du projet:")
print("="*50)
print(f"Nom: {config['project']['name']}")
print(f"Version: {config['project']['version']}")
print(f"")
print("Paramètres de détection:")
print(f"  - Seuil EAR: {config['detection']['eye_aspect_ratio_threshold']}")
print(f"  - Seuil MAR: {config['detection']['mouth_aspect_ratio_threshold']}")
print(f"  - Seuil PERCLOS: {config['detection']['perclos_threshold']*100}%")
print(f"")
print("Dimensions des images:")
print(f"  - Yeux: {config['preprocessing']['image_size_eye']}")
print(f"  - Bouche: {config['preprocessing']['image_size_yawn']}")
print(f"  - Transfer Learning: {config['preprocessing']['image_size_transfer']}")

## 3. Exploration du Dataset

### 3.1 Structure des Données

In [ ]:
# Chemins des données
DATA_RAW = '../data/raw'
DATA_PROCESSED = '../data/processed'

# Exploration de la structure
def explore_directory(path, level=0):
    """Explore récursivement un répertoire."""
    items = []
    if os.path.exists(path):
        for item in os.listdir(path):
            item_path = os.path.join(path, item)
            indent = "  " * level
            if os.path.isdir(item_path):
                num_files = len([f for f in os.listdir(item_path) if os.path.isfile(os.path.join(item_path, f))])
                print(f"{indent}📁 {item}/ ({num_files} fichiers)")
                explore_directory(item_path, level + 1)
            else:
                items.append(item)
    return items

print("Structure des données brutes:")
print("="*50)
if os.path.exists(DATA_RAW):
    explore_directory(DATA_RAW)
else:
    print(f"⚠️  Le dossier {DATA_RAW} n'existe pas encore.")
    print("Créez la structure ou téléchargez les datasets.")

### 3.2 Génération de Données Synthétiques (Démonstration)

In [ ]:
# Création de données synthétiques pour démonstration
# Dans un vrai projet, vous utiliserez des datasets réels

def generate_synthetic_eye_data(n_samples=1000, img_size=(48, 48)):
    """
    Génère des données synthétiques d'yeux pour démonstration.
    """
    np.random.seed(42)
    
    images = []
    labels = []
    
    for i in range(n_samples):
        # Image de base (niveaux de gris)
        img = np.random.randint(0, 50, size=(*img_size, 1), dtype=np.uint8)
        
        # Classe: 0 = ouvert, 1 = fermé
        label = np.random.randint(0, 2)
        
        if label == 0:
            # Œil ouvert: deux cercles (iris + pupille)
            center = (img_size[1]//2, img_size[0]//2)
            cv2.circle(img, center, 15, (200,), -1)  # Iris
            cv2.circle(img, center, 7, (50,), -1)    # Pupille
        else:
            # Œil fermé: ligne horizontale
            y = img_size[0]//2
            cv2.line(img, (10, y), (img_size[1]-10, y), (150,), 3)
        
        # Ajout de bruit
        noise = np.random.randint(-20, 20, size=img.shape, dtype=np.int16)
        img = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)
        
        images.append(img)
        labels.append(label)
    
    return np.array(images), np.array(labels)

# Génération des données
print("Génération de données synthétiques...")
X_eyes, y_eyes = generate_synthetic_eye_data(n_samples=1000)

print(f"✓ Données générées:")
print(f"  - Forme: {X_eyes.shape}")
print(f"  - Classes: {np.bincount(y_eyes)}")
print(f"  - Distribution: {np.bincount(y_eyes)[0]} ouverts, {np.bincount(y_eyes)[1]} fermés")

### 3.3 Visualisation des Données

In [ ]:
# Visualisation des échantillons
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

# Œils ouverts
open_indices = np.where(y_eyes == 0)[0][:5]
for i, idx in enumerate(open_indices):
    axes[0, i].imshow(X_eyes[idx].squeeze(), cmap='gray')
    axes[0, i].set_title(f'Ouvert #{idx}')
    axes[0, i].axis('off')

# Œils fermés
closed_indices = np.where(y_eyes == 1)[0][:5]
for i, idx in enumerate(closed_indices):
    axes[1, i].imshow(X_eyes[idx].squeeze(), cmap='gray')
    axes[1, i].set_title(f'Fermé #{idx}')
    axes[1, i].axis('off')

axes[0, 0].set_ylabel('Yeux Ouverts', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Yeux Fermés', fontsize=12, fontweight='bold')

plt.suptitle('Échantillons de Données Synthétiques - Yeux', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../reports/figures/data_samples.png', dpi=150, bbox_inches='tight')
plt.show()

### 3.4 Distribution des Classes

In [ ]:
# Distribution des classes
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Bar plot
classes = ['Ouvert (0)', 'Fermé (1)']
counts = np.bincount(y_eyes)
colors = ['#2ecc71', '#e74c3c']

bars = axes[0].bar(classes, counts, color=colors, edgecolor='black', linewidth=1.5)
axes[0].set_ylabel('Nombre d''échantillons')
axes[0].set_title('Distribution des Classes')
axes[0].grid(axis='y', alpha=0.3)

# Ajouter les valeurs sur les barres
for bar, count in zip(bars, counts):
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height,
                f'{count}\n({count/len(y_eyes)*100:.1f}%)',
                ha='center', va='bottom', fontsize=11, fontweight='bold')

# Pie chart
axes[1].pie(counts, labels=classes, colors=colors, autopct='%1.1f%%',
          startangle=90, explode=(0.02, 0.02))
axes[1].set_title('Répartition des Classes')

plt.tight_layout()
plt.savefig('../reports/figures/class_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Prétraitement des Images

### 4.1 Initialisation du Préprocesseur

In [ ]:
# Initialisation du préprocesseur
preprocessor = ImagePreprocessor(config_path='../config.yaml')

print("Préprocesseur initialisé:")
print(f"  - Taille yeux: {preprocessor.eye_size}")
print(f"  - Taille bouche: {preprocessor.yawn_size}")
print(f"  - Méthode normalisation: {preprocessor.normalize_method}")

### 4.2 Démonstration du Pipeline de Prétraitement

In [ ]:
# Démonstration sur un échantillon
sample_idx = 0
sample_image = X_eyes[sample_idx]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Original
axes[0, 0].imshow(sample_image.squeeze(), cmap='gray')
axes[0, 0].set_title('Image Originale', fontweight='bold')
axes[0, 0].axis('off')

# Normalisation simple
normalized = sample_image.astype(np.float32) / 255.0
axes[0, 1].imshow(normalized.squeeze(), cmap='gray', vmin=0, vmax=1)
axes[0, 1].set_title('Normalisée [0,1]', fontweight='bold')
axes[0, 1].axis('off')

# Redimensionnée
resized = preprocessor.resize_image(sample_image.squeeze(), (48, 48))
axes[0, 2].imshow(resized, cmap='gray')
axes[0, 2].set_title('Redimensionnée (48x48)', fontweight='bold')
axes[0, 2].axis('off')

# Pipeline complet
preprocessed = preprocessor.preprocess_eye(sample_image)
axes[1, 0].imshow(preprocessed.squeeze(), cmap='gray', vmin=0, vmax=1)
axes[1, 0].set_title('Pipeline Complet', fontweight='bold')
axes[1, 0].axis('off')

# Data Augmentation
augmented = preprocessor.apply_augmentation(sample_image)
axes[1, 1].imshow(augmented.squeeze(), cmap='gray')
axes[1, 1].set_title('Augmentée', fontweight='bold')
axes[1, 1].axis('off')

# Histogramme des pixels
axes[1, 2].hist(sample_image.ravel(), bins=50, color='blue', alpha=0.7, label='Original')
axes[1, 2].hist((preprocessed * 255).ravel(), bins=50, color='red', alpha=0.5, label='Normalisée')
axes[1, 2].set_xlabel('Valeur des pixels')
axes[1, 2].set_ylabel('Fréquence')
axes[1, 2].set_title('Distribution des Pixels', fontweight='bold')
axes[1, 2].legend()

plt.suptitle('Pipeline de Prétraitement', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('../reports/figures/preprocessing_pipeline.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Forme originale: {sample_image.shape}")
print(f"Forme prétraitée: {preprocessed.shape}")
print(f"Valeurs min/max: [{preprocessed.min():.3f}, {preprocessed.max():.3f}]")

## 5. Extraction des Métriques (EAR, MAR)

### 5.1 Définition et Calcul de l'EAR

In [ ]:
# Démonstration du calcul EAR

# Landmarks d'un œil ouvert (forme d'amande)
eye_open_landmarks = [
    (30, 25),   # P1: coin externe
    (25, 20),   # P2: haut gauche
    (35, 20),   # P3: haut droit
    (30, 35),   # P4: coin interne
    (25, 40),   # P5: bas gauche
    (35, 40),   # P6: bas droit
]

# Landmarks d'un œil fermé (ligne)
eye_closed_landmarks = [
    (30, 30),   # P1
    (28, 28),   # P2
    (32, 28),   # P3
    (30, 30),   # P4
    (28, 32),   # P5
    (32, 32),   # P6
]

# Calcul des EAR
ear_open = calculate_ear(eye_open_landmarks)
ear_closed = calculate_ear(eye_closed_landmarks)

print("Calcul de l'Eye Aspect Ratio (EAR):")
print("="*50)
print(f"EAR œil ouvert: {ear_open:.3f}")
print(f"EAR œil fermé: {ear_closed:.3f}")
print(f"Différence: {ear_open - ear_closed:.3f}")
print(f"")
print(f"Seuil typique: ~0.25")
print(f"Si EAR < 0.25 → Œil considéré comme fermé")

### 5.2 Visualisation de l'EAR

In [ ]:
# Visualisation des landmarks et calcul EAR
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

def plot_eye_with_landmarks(ax, landmarks, title, ear_value):
    """Dessine un œil avec ses landmarks."""
    # Fond
    ax.set_xlim(15, 45)
    ax.set_ylim(45, 15)
    ax.set_aspect('equal')
    ax.set_facecolor('#f0f0f0')
    
    # Points
    xs = [p[0] for p in landmarks]
    ys = [p[1] for p in landmarks]
    
    # Dessiner les connexions
    # Ligne horizontale
    ax.plot([landmarks[0][0], landmarks[3][0]], 
           [landmarks[0][1], landmarks[3][1]], 
           'b-', linewidth=2, label='Horizontal')
    # Lignes verticales
    ax.plot([landmarks[1][0], landmarks[5][0]], 
           [landmarks[1][1], landmarks[5][1]], 
           'r--', linewidth=2, label='Vertical 1')
    ax.plot([landmarks[2][0], landmarks[4][0]], 
           [landmarks[2][1], landmarks[4][1]], 
           'r--', linewidth=2, label='Vertical 2')
    
    # Points
    for i, (x, y) in enumerate(landmarks):
        color = 'green' if i in [0, 3] else 'red'
        ax.plot(x, y, 'o', markersize=10, color=color)
        ax.annotate(f'P{i+1}', (x, y), textcoords="offset points", 
                   xytext=(5, 5), fontsize=10, fontweight='bold')
    
    ax.set_title(f'{title}\nEAR = {ear_value:.3f}', fontsize=12, fontweight='bold')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)

plot_eye_with_landmarks(axes[0], eye_open_landmarks, 'ŒIL OUVERT', ear_open)
plot_eye_with_landmarks(axes[1], eye_closed_landmarks, 'ŒIL FERMÉ', ear_closed)

plt.suptitle('Calcul de l''Eye Aspect Ratio (EAR)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../reports/figures/ear_calculation.png', dpi=150, bbox_inches='tight')
plt.show()

### 5.3 Simulation Temps Réel des Métriques

In [ ]:
# Simulation d'une séquence temporelle
np.random.seed(42)

n_frames = 300  # 10 secondes à 30fps
ear_values = []
fatigue_states = []

# Simulation: conducteur de plus en plus fatigué
for i in range(n_frames):
    if i < 100:
        # Éveillé
        ear = np.random.normal(0.35, 0.03)
        state = 'awake'
    elif i < 200:
        # Somnolent (clignements plus longs)
        if 120 < i < 130 or 160 < i < 175:
            ear = np.random.normal(0.15, 0.02)  # Yeux fermés
        else:
            ear = np.random.normal(0.32, 0.04)
        state = 'drowsy'
    else:
        # Très fatigué (micro-siestes)
        if 210 < i < 230 or 250 < i < 280:
            ear = np.random.normal(0.12, 0.02)  # Micro-sieste
        else:
            ear = np.random.normal(0.30, 0.05)
        state = 'very_drowsy'
    
    ear_values.append(max(0.1, min(0.5, ear)))
    fatigue_states.append(state)

ear_values = np.array(ear_values)

# Visualisation
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# EAR au fil du temps
axes[0].plot(ear_values, linewidth=1.5, color='blue', label='EAR')
axes[0].axhline(y=0.25, color='r', linestyle='--', linewidth=2, label='Seuil (0.25)')
axes[0].fill_between(range(n_frames), 0, 0.25, alpha=0.2, color='red', label='Zone fatigue')

# Colorer les phases
axes[0].axvspan(0, 100, alpha=0.1, color='green', label='Phase éveillé')
axes[0].axvspan(100, 200, alpha=0.1, color='orange')
axes[0].axvspan(200, 300, alpha=0.1, color='red')

axes[0].set_ylabel('EAR', fontsize=12)
axes[0].set_title('Simulation: Évolution de l''EAR chez un conducteur fatigué', 
                 fontsize=14, fontweight='bold')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim(0, 0.5)

# Calcul du PERCLOS glissant
window_size = 30
perclos_values = []
for i in range(n_frames):
    start = max(0, i - window_size)
    window = ear_values[start:i+1]
    perclos = np.sum(window < 0.25) / len(window) if len(window) > 0 else 0
    perclos_values.append(perclos)

axes[1].plot(perclos_values, linewidth=1.5, color='purple', label='PERCLOS')
axes[1].axhline(y=0.15, color='r', linestyle='--', linewidth=2, label='Seuil PERCLOS (15%)')
axes[1].fill_between(range(n_frames), 0.15, 1, alpha=0.2, color='red')

# Colorer les phases
axes[1].axvspan(0, 100, alpha=0.1, color='green')
axes[1].axvspan(100, 200, alpha=0.1, color='orange')
axes[1].axvspan(200, 300, alpha=0.1, color='red')

axes[1].set_xlabel('Frames (30 fps)', fontsize=12)
axes[1].set_ylabel('PERCLOS', fontsize=12)
axes[1].set_title('PERCLOS (Pourcentage de fermeture des yeux)', fontsize=12, fontweight='bold')
axes[1].legend(loc='upper left')
axes[1].grid(True, alpha=0.3)

# Annotations des phases
axes[1].text(50, 0.8, 'ÉVEILLÉ', fontsize=12, fontweight='bold', 
            color='green', ha='center')
axes[1].text(150, 0.8, 'SOMNOLENT', fontsize=12, fontweight='bold', 
            color='orange', ha='center')
axes[1].text(250, 0.8, 'DANGER', fontsize=12, fontweight='bold', 
            color='red', ha='center')

plt.tight_layout()
plt.savefig('../reports/figures/ear_timeseries.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Préparation des Données pour l'Entraînement

### 6.1 Division Train/Validation/Test

In [ ]:
from sklearn.model_selection import train_test_split

# Prétraitement de toutes les images
print("Prétraitement des données...")
X_processed = []
for img in tqdm(X_eyes, desc="Prétraitement"):
    processed = preprocessor.preprocess_eye(img)
    X_processed.append(processed)

X_processed = np.array(X_processed)

# Division train/val/test
# D'abord: train (70%) et temp (30%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X_processed, y_eyes, 
    test_size=0.30, 
    random_state=42, 
    stratify=y_eyes
)

# Ensuite: val (15%) et test (15%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.50,
    random_state=42,
    stratify=y_temp
)

print("\nDivision des données:")
print("="*50)
print(f"Train: {X_train.shape[0]} échantillons ({X_train.shape[0]/len(X_processed)*100:.1f}%)")
print(f"Validation: {X_val.shape[0]} échantillons ({X_val.shape[0]/len(X_processed)*100:.1f}%)")
print(f"Test: {X_test.shape[0]} échantillons ({X_test.shape[0]/len(X_processed)*100:.1f}%)")
print(f"\nForme des données: {X_train.shape}")

# Distribution des classes
print("\nDistribution des classes:")
print(f"Train: {np.bincount(y_train)}")
print(f"Val: {np.bincount(y_val)}")
print(f"Test: {np.bincount(y_test)}")

### 6.2 Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Configuration de l'augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)

# Visualisation d'exemples augmentés
fig, axes = plt.subplots(3, 5, figsize=(15, 9))

sample = X_train[0:1]  # Un échantillon

# Image originale
axes[0, 0].imshow(sample[0].squeeze(), cmap='gray')
axes[0, 0].set_title('Originale', fontweight='bold')
axes[0, 0].axis('off')

# Générer des variations
augmented_images = []
aug_iter = datagen.flow(sample, batch_size=1)

for i in range(14):
    aug_img = next(aug_iter)[0]
    augmented_images.append(aug_img)

# Afficher toutes les images
for idx, img in enumerate(augmented_images, 1):
    row = idx // 5
    col = idx % 5
    axes[row, col].imshow(img.squeeze(), cmap='gray')
    axes[row, col].set_title(f'Augmentée #{idx}', fontweight='bold')
    axes[row, col].axis('off')

plt.suptitle('Exemples de Data Augmentation', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../reports/figures/data_augmentation.png', dpi=150, bbox_inches='tight')
plt.show()

### 6.3 Sauvegarde des Données Préparées

In [ ]:
# Sauvegarde des données préparées
import os

processed_dir = '../data/processed'
os.makedirs(processed_dir, exist_ok=True)

# Sauvegarde
np.save(f'{processed_dir}/X_train.npy', X_train)
np.save(f'{processed_dir}/X_val.npy', X_val)
np.save(f'{processed_dir}/X_test.npy', X_test)
np.save(f'{processed_dir}/y_train.npy', y_train)
np.save(f'{processed_dir}/y_val.npy', y_val)
np.save(f'{processed_dir}/y_test.npy', y_test)

print("✓ Données sauvegardées:")
print(f"  - {processed_dir}/X_train.npy ({X_train.shape})")
print(f"  - {processed_dir}/X_val.npy ({X_val.shape})")
print(f"  - {processed_dir}/X_test.npy ({X_test.shape})")
print(f"  - Labels correspondants")

## 7. Résumé et Prochaines Étapes

### 7.1 Résumé de l'Exploration

In [ ]:
# Résumé visuel
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Titre principal
fig.suptitle('RÉSUMÉ DE L''EXPLORATION DES DONNÉES', 
            fontsize=18, fontweight='bold', y=0.98)

# 1. Échantillons
ax1 = fig.add_subplot(gs[0, 0])
ax1.imshow(X_eyes[0].squeeze(), cmap='gray')
ax1.set_title('Œil Ouvert', fontweight='bold')
ax1.axis('off')

ax2 = fig.add_subplot(gs[0, 1])
ax2.imshow(X_eyes[closed_indices[0]].squeeze(), cmap='gray')
ax2.set_title('Œil Fermé', fontweight='bold')
ax2.axis('off')

# 2. Distribution
ax3 = fig.add_subplot(gs[0, 2])
ax3.pie(np.bincount(y_eyes), labels=['Ouvert', 'Fermé'], 
       colors=['#2ecc71', '#e74c3c'], autopct='%1.1f%%')
ax3.set_title('Distribution', fontweight='bold')

# 3. EAR au fil du temps
ax4 = fig.add_subplot(gs[1, :])
ax4.plot(ear_values[:200], linewidth=1.5, color='blue')
ax4.axhline(y=0.25, color='r', linestyle='--', linewidth=2)
ax4.fill_between(range(200), 0, 0.25, alpha=0.2, color='red')
ax4.set_ylabel('EAR')
ax4.set_title('Simulation EAR', fontweight='bold')
ax4.grid(True, alpha=0.3)

# 4. Split des données
ax5 = fig.add_subplot(gs[2, :2])
splits = ['Train', 'Validation', 'Test']
sizes = [len(X_train), len(X_val), len(X_test)]
colors = ['#3498db', '#2ecc71', '#e74c3c']
bars = ax5.bar(splits, sizes, color=colors, edgecolor='black')
ax5.set_ylabel('Nombre d''échantillons')
ax5.set_title('Division Train/Validation/Test', fontweight='bold')
for bar, size in zip(bars, sizes):
    ax5.text(bar.get_x() + bar.get_width()/2., bar.get_height(),
            f'{size}\n({size/len(X_processed)*100:.0f}%)', 
            ha='center', va='bottom', fontweight='bold')

# 5. Info
ax6 = fig.add_subplot(gs[2, 2])
ax6.axis('off')
info_text = f"""
📊 STATISTIQUES
━━━━━━━━━━━━━━━

Total échantillons: {len(X_processed)}

Dimensions images:
  {X_processed.shape[1]} × {X_processed.shape[2]}

Canaux: {X_processed.shape[3]}

Normalisation: [0, 1]

Classes: 2 (binaire)
"""
ax6.text(0.1, 0.5, info_text, fontsize=11, family='monospace',
        verticalalignment='center', bbox=dict(boxstyle='round', 
        facecolor='wheat', alpha=0.5))

plt.savefig('../reports/figures/exploration_summary.png', dpi=150, bbox_inches='tight')
plt.show()

### 7.2 Prochaines Étapes

Le notebook suivant (02_modelisation_cnn.ipynb) couvrira:

1. **Construction du CNN** - Architecture du modèle
2. **Entraînement** - Descente de gradient, callbacks
3. **Évaluation** - Métriques de performance
4. **Transfer Learning** - Fine-tuning de MobileNetV2

### Concepts du Cours Appliqués

- ✅ **Chapitre 1**: Normalisation, prétraitement
- ✅ **Chapitre 2**: Préparation des données, split train/val/test
- ✅ **Chapitre 3-4**: Concepts de CNN (à venir)

---

**Notebook 01 terminé!** 🎉